<a href="https://colab.research.google.com/github/fsilvao/ia/blob/main/Arquitectura_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import math
import torch
import torch.nn as nn
import numpy as np


class Generador(nn.Module):
  def __init__(self, input_length: int):
    super(Generador, self).__init__()
    self.dense_layer = nn.Linear(int(input_length), int(input_length))
    self.activation = nn.Sigmoid()

  def forward(self,x):
    return self.activation(self.dense_layer(x))


class Discriminador(nn.Module):
  def __init__(self, input_length:int):
    super(Discriminador, self).__init__()
    self.dense = nn.Linear(int(input_length), 1)
    self.activation =nn.Sigmoid()

  def forward(self,x):
    return self.activation(self.dense(x))


def create_binary_list_from_int(number:int):
  #Convierte un entero en su representación binaria con 7 dígitos
  if number <0 or type(number) is not int:
    raise ValueError("Solo se perimeten enteros positivos")
  return [int(x) for x in list(bin(number))[2:]]

def generate_even_data(max_int: int, batch_size: int = 16):
  #Función que genera datos de números pares
  #Obtener el número de lugares binarios necesarios para representar el número más alto
  max_length = int(math.log(max_int, 2))

  #Muestrar un número igual a batch_size  de enteros en el rango 0 - max_int
  sampled_integers = np.random.randint(0, int(max_int /2), batch_size)

  #Crea una lista de etiquetas (todas 1) ya que todos los números generados aquí son pares/even
  labels = [1] * batch_size

  #Crear una lista de números binarios para el entrenamiento
  data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
  data =[([0] * (max_length - len(x))) + x for x in data]

  return labels, data

def convert_float_matrix_to_int_list(float_matrix: np.array, threshold: float =0.5):
  #Convierte un set de valores en binario a números enteros
  return [int("".join([str(int(y)) for y in x]), 2) for x in float_matrix >= threshold]


def train(max_int: int = 128, batch_size: int = 16, training_steps: int = 500, learning_rate: float = 0.001, print_output_every_n_steps: int = 10):
  #El entrenamiento de la GAN para aprender a generar y distinguir números pares
  input_length = int(math.log(max_int, 2))

  #Creamos los modelos
  generador= Generador(input_length=input_length) #1.- instanciar un generador con argumento igual al tamaño/longitud del input
  discriminador = Discriminador(input_length=input_length) #2.- instanciar un discriminador con argumento igual al tamaño/longitud del input

  #Optimizers
  generator_optimizer = torch.optim.Adam(generador.parameters(), lr=learning_rate)
  discriminator_optimizer = torch.optim.Adam(discriminador.parameters(), lr=learning_rate)

  #loss
  loss = nn.BCELoss() #3.- Qué función de pérdida deberíamos utilizar? defínala acá

  for i in range(training_steps):
    # 4.- Poner los gradientes en cero del generador y el discriminador
    generator_optimizer.zero_grad()
    discriminator_optimizer.zero_grad()

    #Creamos un input de ruido para el generador
    noise = torch.randint(0,2, size=(batch_size, input_length)).float()

    generated_data = generador(noise) # 5.- generated_data es lo que sale del generador al pasarle como entrada el ruido/noise

    #Generamos ejemplos de datos reales (números pares)
    true_labels, true_data = generate_even_data(max_int, batch_size = batch_size)
    true_labels = torch.tensor(true_labels).float()
    true_data = torch.tensor(true_data).float()


    #Primero entrenamos el generador
    #Invertimos las etiquetas y no entrenamos el discriminador, porque queremos que el generador
    #genere datos que el discriminador clasifique como real
    generator_discriminator_out = discriminador(generated_data).squeeze()
    generator_loss = loss(generator_discriminator_out, true_labels)
    generator_loss.backward()
    generator_optimizer.step()

    #Luego entrenamos el discriminador, primero con datos reales y luego con datos generados, finalmente calculo el promedio
    true_discriminator_out = discriminador(true_data).squeeze()
    true_discriminator_loss = loss(true_discriminator_out, true_labels)

    #Ahora pasamos los datos gnerados a través del discriminador, agregando .detach() para que no "entrene" el generador
    generator_discriminator_out = discriminador(generated_data.detach()).squeeze()
    generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size))

    discriminator_loss = (true_discriminator_loss + generator_discriminator_loss)/2
    discriminator_loss.backward()
    discriminator_optimizer.step()
    if i % print_output_every_n_steps == 0:
      print("Step ", i," : ", convert_float_matrix_to_int_list(generated_data))

  return generador, discriminador


max_int = 128
batch_size = 16
training_steps = 1000
learning_rate = 0.001
input_length = int(math.log(max_int, 2))

generador, discriminador = train(max_int = max_int, batch_size = batch_size, training_steps = training_steps, learning_rate= learning_rate, print_output_every_n_steps=100)
noise = torch.randint(0,2, size=(batch_size, input_length)).float()
print("Ruido inicial: ", noise)
for idx, num in enumerate(convert_float_matrix_to_int_list(noise)):
  print("Número generado es: ", num)

generated_data= generador(noise)

#Evaluación del modelo
print("Evaluando datos generados por el generador: ")
evaluated_data = discriminador(generated_data).squeeze()

for idx, num in enumerate(convert_float_matrix_to_int_list(generated_data)):
  print("Número generado es: ", num)
  print("Discriminador dice: ", evaluated_data[idx].item())










Step  0  :  [103, 111, 87, 103, 87, 39, 103, 111, 111, 7, 87, 39, 71, 71, 7, 1]
Step  100  :  [111, 103, 103, 39, 103, 111, 103, 47, 47, 39, 111, 39, 39, 47, 39, 103]
Step  200  :  [47, 47, 111, 47, 47, 47, 111, 47, 47, 47, 47, 47, 47, 111, 47, 47]
Step  300  :  [47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47]
Step  400  :  [47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47]
Step  500  :  [47, 47, 47, 47, 47, 47, 47, 46, 47, 47, 47, 47, 47, 47, 47, 47]
Step  600  :  [46, 46, 46, 46, 46, 47, 46, 46, 46, 47, 46, 46, 46, 47, 46, 46]
Step  700  :  [46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46]
Step  800  :  [62, 46, 62, 46, 46, 46, 46, 62, 62, 46, 62, 46, 46, 62, 62, 62]
Step  900  :  [62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62]
Ruido inicial:  tensor([[1., 0., 1., 0., 0., 1., 0.],
        [0., 1., 0., 1., 0., 1., 1.],
        [1., 1., 1., 0., 0., 1., 0.],
        [1., 0., 1., 1., 1., 1., 0.],
        [1., 1., 0., 1., 1., 1